<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [14]</a>'.</span>

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
import pandas as pd
import os
import sys

PYSPARK_PYTHON = os.getenv("PYSPARK_PYTHON") 
PYSPARK_DRIVER_PYTHON = os.getenv("PYSPARK_DRIVER_PYTHON")

In [2]:
import pyspark
from delta import configure_spark_with_delta_pip, DeltaTable
import json

# Load the configuration JSON file
with open('/usr/local/spark/conf/spark-defaults.json', 'r') as f:
    config = json.load(f)

# Initialize the Spark session builder
builder = pyspark.sql.SparkSession.builder.appName("MyApp1").config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog").config("spark.pyspark.python", PYSPARK_PYTHON)\
    .config("spark.pyspark.driver.python", PYSPARK_DRIVER_PYTHON)

# Read the packages from the text file
packages = []
with open('/usr/local/spark/conf/packages.txt', 'r') as file:
    # Read each line and strip newlines or extra spaces
    packages = [line.strip() for line in file if line.strip()]

# # Add packages to the Spark session configuration
builder.config("spark.jars.packages", ",".join(packages))

# Apply the configurations from the JSON file to the Spark session
for key, value in config.items():
    builder.config(key, value)

In [3]:

# Configure Spark with Delta Lake (if needed)
spark = configure_spark_with_delta_pip(builder).getOrCreate()
# Now you can use the Spark session
spark

In [4]:
trgt_path_processed = "/mnt/Calendar/Calendar_Parquet/"
trgt_path_csv = "/mnt/Calendar/Calendar_Processed/"

In [5]:
# Create a DataFrame with date range
start_date = "2000-01-01"
end_date = "2050-12-31"

In [6]:
# Create a DataFrame with a single row containing the start and end date
date_range_df = spark.createDataFrame([(start_date, end_date)], ["start_date", "end_date"])

# Generate date sequence
date_sequence_df = date_range_df.select(
    sequence(
        to_date(date_range_df.start_date).alias("start_date"),
        to_date(date_range_df.end_date).alias("end_date")
    ).alias("date")
)

In [7]:
# Explode the sequence into separate rows
df_date = date_sequence_df.selectExpr("explode(date) as date")

In [8]:
df_output = df_date.withColumn("DateSK", regexp_replace("date", "-", "")).withColumn("Year", year("date"))\
    .withColumn("Month",date_format("date","MMMM")).withColumn("Quarter",concat(year("date"), lit(" Q"), quarter("date")))

In [9]:
print(df_output.count())

18628


In [10]:
df_output.createOrReplaceTempView("vw_source")

In [11]:
if DeltaTable.isDeltaTable(spark, trgt_path_processed):
    column_name = df_output.columns
    set_clause = ", ".join([f"target.{i} = source.{i}" for i in column_name])
    query = f"""MERGE INTO delta.`{trgt_path_processed}` AS target USING vw_source AS source ON target.DateSK = source.DateSK WHEN MATCHED THEN UPDATE SET {set_clause}"""
else:
    query=f"""CREATE TABLE delta.`{trgt_path_processed}` USING DELTA AS SELECT * FROM vw_source"""
print(query)
spark.sql(query)

MERGE INTO delta.`/mnt/Calendar/Calendar_Parquet/` AS target USING vw_source AS source ON target.DateSK = source.DateSK WHEN MATCHED THEN UPDATE SET target.date = source.date, target.DateSK = source.DateSK, target.Year = source.Year, target.Month = source.Month, target.Quarter = source.Quarter


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [12]:
# Save the DataFrame to a CSV file
spark.read.format("delta").load(trgt_path_processed) \
    .coalesce(1) \
    .write.format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(trgt_path_csv)

In [13]:
trgt_copy_path = "/mnt/Calendar/Calendar_Processed/calendar.csv"

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [14]:
import shutil
shutil.copy(trgt_path_processed, trgt_copy_path)

IsADirectoryError: [Errno 21] Is a directory: '/mnt/Calendar/Calendar_Parquet/'

In [ ]:
# spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)
# # Create DeltaTable object
# delta_table = DeltaTable.forPath(spark, trgt_path_processed)

# # Vacuum the table
# delta_table.vacuum(0)  # Retain 0 hours to clean up all unreferenced files

In [ ]:
spark.stop()